In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import random
from collections import defaultdict
import pickle
import os

import gurobipy as gp
from gurobipy import GRB

In [ ]:
n = 4 # max dimention of the input vector
n_samples = 10 # number of sampled quadratic function for each dimension

# set random seed
global_seed = 777
random.seed(global_seed)
np.random.seed(global_seed)
seeds = np.random.randint(1,1000,(1,n_samples))[0]

In [ ]:
times = [] # list that stores solution times
objs = [] # list that stores obj function values
num_binvars = [] # list that stores # binary variables
num_constrs = [] # list that stores # constraints
best_models = [] # list that stores the best trained model
mapes = [] # list that stores MAPEs of the trained model

In [ ]:
for i in tqdm(range(n_samples)):
    
    # read data from csv file
    file_name = './power_plant.csv'
    df = pd.read_csv(file_name)
    
    # Y = ReLU_Network(X)
    X_variable_raw = df.iloc[:,:-1].values
    Y_variable_raw = df.iloc[:,-1].values
    X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_variable_raw, Y_variable_raw,
                                                                        random_state=seeds[i], test_size = 0.25)
    
    # scaling
    sc_X = MinMaxScaler()
    sc_y = MinMaxScaler()

    X_variable = sc_X.fit_transform(X_train_raw)
    Y_variable = sc_y.fit_transform(y_train_raw.reshape(-1,1))
    Y_variable = np.squeeze(Y_variable)
    
    labels=np.array(Y_variable)
    features = np.array(X_variable)
    
    #training
    random.seed(seeds[i])
    np.random.seed(seeds[i])
    gs_relu = GridSearchCV(MLPRegressor(max_iter = 1000),
				  param_grid = {'learning_rate_init': [1e-2, 2e-2, 5e-2, 1e-3, 2e-3, 5e-3, 1e-4],
                                'hidden_layer_sizes': [(10, 5), (20, 10), (40, 20), (50, 20), (50, 30, 30)]},
				  return_train_score = True, cv = 5,
				  scoring = 'neg_mean_squared_error',
                  verbose = 0)
    gs_relu.fit(features, labels)
    nn = gs_relu.best_estimator_
    best_models.append(nn)
    
    # prediction accuracy 
    y_predict = nn.predict(sc_X.transform(X_test_raw))
    y_predict_inversed = sc_y.inverse_transform(y_predict.reshape(-1,1))
    mapes.append(mean_absolute_percentage_error(y_test_raw, y_predict_inversed))
    
    # obtain MILP parameters from the ReLU network structure/parameters
    # weight and bias
    weights = nn.coefs_
    biases = nn.intercepts_
    
    # kj_raw: number of neurons in the ith hidden layer; k_hidden: number of hidden layers
    kj_raw = nn.hidden_layer_sizes
    k_hidden = nn.n_layers_ - 2
    
    # indices_k_hidden: set of hidden layers; indices_kj_hidden: set of tuple (layer k, jth neuron in this layer)
    # indices_kj_dict: mapping of hidden layer k -> list of neurons j in this hidden layer
    indices_k_hidden = range(1, k_hidden+1)
    indices_kj_hidden = []
    indices_kj_dict = defaultdict(list)
    for k in indices_k_hidden:
        for j in range(kj_raw[k-1]):
            indices_kj_hidden.append((k,j))
            indices_kj_dict[k].append(j)
    indices_kj_dict[nn.n_layers_ - 1] = [0]
    indices_kj_dict[0] = list(range(n))
    # indices_kj_first: (k, j) tuple for input layer; indices_kj_last: (k, j) tuple for the output layer
    # indices_kj: (k, j) tuple for all the layers in the network
    # indices_kj_nofirst: (k, j) tuple for all the layers except for the first layer      
    indices_kj_first = [(0,i) for i in range(n)]
    indices_kj_last = [(nn.n_layers_ - 1, 0)]
    indices_kj = indices_kj_first + indices_kj_hidden + indices_kj_last
    indices_kj_nofirst = indices_kj_hidden + indices_kj_last
    
    # paramters
    
    # lower/upper bound of variable X
    # x_lb_first/x_ub_first: lower/upper bound of variable X for the input layer
    x_lb_first = np.amin(X_variable_raw, axis = 0).tolist()
    x_ub_first = np.amax(X_variable_raw, axis = 0).tolist()
    
    # x_ub/x_lb: lower/upper bound of variable X in all the layers
    # x_ub_dict: mapping of layer -> list of upper bound of all neurons in the layer
    bigM = 1e4
    x_ub = x_ub_first + [bigM] * len(indices_kj_hidden) + [bigM]
    x_lb = x_lb_first + [0] * len(indices_kj_hidden) + [0]
    x_ub_dict = defaultdict(list)
    x_ub_dict[0] = x_ub_first
    for k in indices_k_hidden:
        x_ub_dict[k] = [bigM] * kj_raw[k-1]
    x_ub_dict[k_hidden + 1] = [bigM]
    
    # scaling: the network is trained using the min-max scaled variables, so we need to have scaler_coeff to do
    # (1) min_max scaling the input variable (happened at the input layer)
    # (2) min_max scaling the output variable (happened at the output layer)
    # to make the constraints consistent we still have scaler_coeff[k] for hidden layer but the values are set to 1 (no scaling)
    scaler_coeff = {}
    scaler_coeff[0] = np.reciprocal(sc_X.data_range_).tolist() # scaler_coeff = 1/(Xmax - Xmin) (i.e., the original range)
    for k in indices_k_hidden:
        scaler_coeff[k] = [1] * kj_raw[k-1] # scaler_coeff = 1 (no scaling)
    scaler_coeff[k_hidden + 1] = np.reciprocal(sc_y.data_range_).tolist() # scaler_coeff = 1/(Ymax - Ymin)
    
    scaler_min = {}
    scaler_min[0] = sc_X.data_min_.tolist() # Xmin before scaling
    for k in indices_k_hidden:
        scaler_min[k] = [0] * kj_raw[k-1] # Xmin = 0 (no scaling)
    scaler_min[k_hidden + 1] = sc_y.data_min_.tolist() # Ymin before scaling
    
    # create a new model
    m = gp.Model("RELU")
    m.Params.LogToConsole = 0
    m.setParam('TimeLimit', 60)

    # create variables
    z = m.addVars(indices_kj_nofirst, name = 'z', vtype = GRB.BINARY)
    x = m.addVars(indices_kj, ub = x_ub, lb = x_lb, name = 'x')
    s = m.addVars(indices_kj_nofirst, name = 's')
    
    # maximization
    m.setObjective(x[nn.n_layers_-1,0], GRB.MAXIMIZE)

    # add constraint
    m.addConstrs((gp.quicksum(weights[k-1][l][j] * (x[k-1,l] - scaler_min[k-1][l]) * scaler_coeff[k-1][l] for l in indices_kj_dict[k-1]) + biases[k-1][j] \
                  == (x[k,j] - scaler_min[k][j]) * scaler_coeff[k][j] - s[k,j] for (k,j) in indices_kj_nofirst),
                 name = 'calc_hidden_layers')
    m.addConstrs((x[k,j] <= x_ub_dict[k][j] * z[k,j] for (k,j) in indices_kj_nofirst), name = "constraint_x")
    m.addConstrs((s[k,j] <= x_ub_dict[k][j] * (1 - z[k,j]) for (k,j) in indices_kj_nofirst), name = "constraint_z")
    
    m.update()
    m.write("m.lp")
    
    m.optimize()
    
    times.append(m.Runtime)
    objs.append(m.objVal)
    num_binvars.append(m.NumBinVars)
    num_constrs.append(m.NumConstrs)

In [ ]:
print(times)
print(objs)
print(num_binvars)
print(num_constrs)
print(mapes)

In [ ]:
def pickle_save(path, file, filename):
    file_loc = path + '/' + filename + '.pickle'
    with open(file_loc, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

# create the directory to save the results
path = './results_opt_powerplant_relu'

try:
    os.mkdir(path)
except FileExistsError:
    print('Folder already exists')

In [ ]:
pickle_save(path, objs, 'objs')
pickle_save(path, times, 'times')
pickle_save(path, num_binvars, 'num_binvars')
pickle_save(path, num_constrs, 'num_constrs')
pickle_save(path, mapes, 'mapes')